<a href="https://colab.research.google.com/github/" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 AI Video Upscale Workflow (4K 24fps Standard)

Welcome Huy! This version is optimized for **4K @ 24fps** (Cinematic Standard).

### 📌 Workflow Overview:
1. **Connect Drive**: Mount your Google Drive.
2. **Install Tools**: Setup Real-ESRGAN and RIFE.
3. **Select Film**: Pick a specific file from your Drive.
4. **Render**: Upscale to 4K (and optionally bump to 60fps).
5. **Finalize**: Generate a high-quality **H.265 MKV** with full audio/subs/metadata.

## 1️⃣ Setup & Mounting
Mount Drive and verify you have a **T4 GPU**.

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

!nvidia-smi

## 2️⃣ Install AI Tools
Takes about 2-3 minutes.

In [ ]:
# Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd /content/Real-ESRGAN
!pip install basicsr facexlib gfpgan -q
!pip install -r requirements.txt -q
!python setup.py develop
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/ -q

# RIFE (Optional frame interpolation)
%cd /content
!git clone https://github.com/hzwer/Practical-RIFE.git RIFE
%cd /content/RIFE

# Download RIFE model from Hugging Face (verified working)
!mkdir -p train_log
!wget https://huggingface.co/jbilcke-hf/varnish/resolve/main/rife/flownet.pkl -O train_log/flownet.pkl -q

print("✅ Installation complete!")

## 3️⃣ Select Your Film
Run this to see available files in your input folder.

In [ ]:
INPUT_DIR = "/content/drive/MyDrive/Movies/Input" # @param {type:"string"}
if os.path.exists(INPUT_DIR):
    print("🎬 Danh sách phim trong thư mục:")
    for f in sorted(os.listdir(INPUT_DIR)):
        if f.endswith(('.mp4', '.mkv', '.avi', '.mov')):
            print(f"  - {f}")
else:
    print("❌ Không tìm thấy thư mục! Hãy kiểm tra lại đường dẫn.")

## 4️⃣ Run AI Rendering
Copy the file name from the list above. `ENABLE_60FPS` is **OFF** by default.

In [ ]:
FILE_NAME = "phim_cua_huy.mp4" # @param {type:"string"}
SCALE = 2 # @param {type:"integer"}
ENABLE_60FPS = False # @param {type:"boolean"}

INPUT_PATH = os.path.join(INPUT_DIR, FILE_NAME)
TEMP_UPSCALE = "/content/upscaled_temp.mp4"
TEMP_60FPS = "/content/upscaled_60fps_temp.mp4"

print(f"🚀 Bắt đầu xử lý: {FILE_NAME}")

# --- STEP A: UPSCALE (Real-ESRGAN) ---
print("🎨 Đang Upscale lên 4K...")
%cd /content/Real-ESRGAN
!python inference_realesrgan_video.py -n realesr-animevideov3 -i "{INPUT_PATH}" -o "{TEMP_UPSCALE}" -s {SCALE}

# --- STEP B: INTERPOLATE (RIFE) ---
if ENABLE_60FPS:
    print("✨ Đang nâng lên 60FPS (RIFE)...")
    %cd /content/RIFE
    !python3 inference_video.py --video="{TEMP_UPSCALE}" --output="{TEMP_60FPS}" --exp=1
    FINAL_VIDEO_STREAM = TEMP_60FPS
else:
    print("🎞️ Bỏ qua 60FPS, giữ nguyên 24fps gốc.")
    FINAL_VIDEO_STREAM = TEMP_UPSCALE

## 5️⃣ Finalize: Create MKV (H.265)
Merges AI video with original Audio/Subs/Metadata.

In [ ]:
base_name = FILE_NAME.rsplit('.', 1)[0]
suffix = "_4K_60fps" if ENABLE_60FPS else "_4K"
OUTPUT_NAME = f"{base_name}{suffix}.mkv"

FINAL_OUTPUT_DIR = "/content/drive/MyDrive/Movies/Output" # @param {type:"string"}
FINAL_PATH = os.path.join(FINAL_OUTPUT_DIR, OUTPUT_NAME)

if not os.path.exists(FINAL_OUTPUT_DIR):
    os.makedirs(FINAL_OUTPUT_DIR)

print(f"📦 Đang đóng gói MKV: {FINAL_PATH}")

!ffmpeg -i "{INPUT_PATH}" -i "{FINAL_VIDEO_STREAM}" \
    -map 1:v -map 0:a? -map 0:s? \
    -c:v libx265 -tag:v hvc1 -crf 23 -preset fast \
    -c:a copy -c:s copy \
    -map_metadata 0 \
    "{FINAL_PATH}" -y

print("✅ HOÀN THÀNH!")